# tensorflow使ってみた
深層学習ではなくソフトマックス回帰分析による分類をやる

## インポート

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

## データの取得と整形
pd.read_csv：データをRのデータフレーム型で取得
height, weight, labelの3つのカラム

bclass：辞書クラス

csv["label_pat"]：新しいカラムを追加してる
series.apply ： seriesの各値に関して関数を実行
ここでは[1,0,0],[0,1,0],[0,0,1]のどれかを格納

In [14]:
# 身長,体重,ラベルのCSVデータを読み出す --- (※1)
csv = pd.read_csv("/users/yuya/desktop/src/ch5/bmi.csv")
print(type(csv))
print(csv.shape)

# データを正規化 --- (※2)
csv["height"] = csv["height"] / 200#データを大体0〜1にしてる
csv["weight"] = csv["weight"] / 100

# ラベルを三次元のクラスで表す --- (※3)
# - thin=(1,0,0) / normal=(0,1,0) / fat=(0,0,1)
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}#クラスラベルを数字に
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

<class 'pandas.core.frame.DataFrame'>
(20000, 3)


## テストデータの取得
5000のデータを性能評価用にする
test_pat　：　入力となるデータだけ取得
test_ans : 出力のデータを取得

In [15]:
# 正解率を求めるためにテストデータを準備 --- (※4)
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

# ここからtensorflowのためのデータフローグラフの構築

## 訓練データに対して、
placeholder : 身長と体重の2変数が入る予定の入れ物(型：float32, None:任意のサンプル数, 2 : 2変数)

In [17]:
# データフローグラフを構築する --- (※5)
# データを入れるプレースホルダを宣言
x  = tf.placeholder(tf.float32, [None, 2]) # 身長,体重のデータを入れる
y_ = tf.placeholder(tf.float32, [None, 3]) # 答えのラベルを入れる

## ソフトマックス回帰の定義
y = softmax(Wx + b)
W : (2,3) 重み
x : (2,1)　データ
b : (3,1) バイアス
Wとbは0で初期化
出力は(3,1)の[1,0,0]のようになってほしい

In [19]:
# 変数を宣言 --- (※6)
W = tf.Variable(tf.zeros([2, 3])); # 重み
b = tf.Variable(tf.zeros([3])); # バイアス
# ソフトマックス回帰を定義 --- (※7)
y = tf.nn.softmax(tf.matmul(x, W) + b)

## モデル訓練
reduce_sum : 任意の次元の全ての要素をたす。何も指定しないと1次元の値を返す
train.GradientDescentOptimizer : (optimizerの)学習係数を指定
optimizer.minimize : ()ないの関数を最小にするパラメータを決定

In [20]:
# モデルを訓練する --- (※8)
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))#誤差関数として交差エントロピー
optimizer = tf.train.GradientDescentOptimizer(0.01)#学習係数0.01
train = optimizer.minimize(cross_entropy)

## 正解率
得られたパラメータを用いて入力から予測する
tf.equal : 渡された2つのベクトルが一致しているか

tf.argmax() : 第２パラメータに「1」を入れると、第１パラメータの行ごとの最大の列を返す

reduce_mean : 任意の次元のテンソルの平均を求める
tf.cast : 第1パラメーターを第2パラメーターのデータ・タイプに変換する

In [23]:
# 正解率を求める
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

## 実行
tf.Session()：このコマンドでtensorflowで計算を実行するためのセッションが始まる
run() : 計算を始める
tf.global_variables_initializer：変数の初期化(毎回行う)

In [29]:
# セッションを開始
sess = tf.Session()
#sess.run(tf.initialize_all_variables()) #変数を初期化 旧バージョン
sess.run(tf.global_variables_initializer())

## 上で訓練したパラメータをテストデータに使う
100サンプルずつ3500回学習
データの1番から14101番まで訓練で使う
x_pat : 2つの入力部分のデータだけ取得
y_ans : 出力となる[1,0,0]のようなリストを取得
fd : 辞書

train : 最適化の関数的な
feed_dict : Placeholderは実行時にfeed_dictという仕組みを通じて値を外挿できる仕組み

500ステップごとに誤差関数の値と正解率を返す

test_pat : 取り出しておいたテスト用の(5000,2)の入力
test_ans : 取り出しておいたテスト用の(5000,1)の正解

In [30]:
# テストデータを用いて学習させる
for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    
    sess.run(train, feed_dict=fd)
    
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "cre=", cre, "acc=", acc)

# 最終的な正解率を求める
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("正解率=", acc)

step= 0 cre= 108.663 acc= 0.3242
step= 500 cre= 57.5887 acc= 0.8904
step= 1000 cre= 45.0209 acc= 0.898
step= 1500 cre= 41.6543 acc= 0.9566
step= 2000 cre= 34.664 acc= 0.943
step= 2500 cre= 34.287 acc= 0.9674
step= 3000 cre= 26.8808 acc= 0.9726
正解率= 0.9712


## 正解かどうかを返す
(5000,1)のtrue or false

In [32]:
sess.run(predict, feed_dict={x: test_pat, y_: test_ans})

(5000,)

## ソフトマックス回帰より得られる出力
(5000, 3)の行列

In [33]:
sess.run(y, feed_dict={x: test_pat, y_: test_ans})

array([[  8.83783959e-03,   2.96603352e-01,   6.94558859e-01],
       [  8.04228842e-01,   1.91755563e-01,   4.01561800e-03],
       [  1.04206100e-01,   7.76886463e-01,   1.18907429e-01],
       ..., 
       [  9.95831430e-01,   4.16785572e-03,   7.23445680e-07],
       [  9.56869841e-01,   4.30755280e-02,   5.45924886e-05],
       [  2.36273110e-02,   5.50715029e-01,   4.25657690e-01]], dtype=float32)

## 最大の列番号を返す
つまり、分類結果を表す

In [34]:
sess.run(tf.argmax(y,1), feed_dict={x: test_pat, y_: test_ans})

array([2, 0, 1, ..., 0, 0, 1])